In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from lazypredict.Supervised import LazyRegressor


In [2]:
df = pd.read_csv(r"C:\Users\DHONI HANIF\OneDrive\Documents\AI_Collection_and_Loss_Reverse_Forecast\modelling\kolektor\regresi_for_time\data\kolektor.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1000 non-null   int64  
 1   time_to_collect           1000 non-null   int64  
 2   avg_bill_methods          1000 non-null   object 
 3   debtor_volume_handled     1000 non-null   int64  
 4   bill_amount_collected     1000 non-null   int64  
 5   total_actual              1000 non-null   int64  
 6   total_cost                1000 non-null   int64  
 7   success_rate              1000 non-null   float64
 8   collector_age             1000 non-null   int64  
 9   collector_marital_status  1000 non-null   object 
 10  collector_gender          1000 non-null   object 
 11  negotiation_skills        1000 non-null   object 
 12  collector_experience      1000 non-null   int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 101.7+ KB


In [3]:
for i in df.columns:
    if df[i].dtype == "object":
        df[i] = df[i].astype("category")

df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [4]:
df.head()

,time_to_collect,avg_bill_methods,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,collector_age,collector_marital_status,collector_gender,negotiation_skills,collector_experience
0,0,sms or WA,7,14400000,35700000,277706,40.00,45,Menikah,laki-laki,Baik,4
1,11,panggilan,35,79000000,160500000,22382611,49.00,32,Cerai mati,perempuan,Buruk,0
2,4,sms or WA,6,7100000,26700000,205389,27.00,62,Cerai hidup,laki-laki,Buruk,0
3,29,datang ke tempat,63,782000000,1395000000,482942548,56.00,55,Cerai mati,perempuan,Cukup baik,2
4,22,datang ke tempat,35,346000000,485500000,151035603,71.00,53,Menikah,perempuan,Cukup baik,2


In [5]:
df["avg_bill_methods"].value_counts()

avg_bill_methods
datang ke tempat    481
sms or WA           198
panggilan           162
surat panggilan     159
Name: count, dtype: int64

In [6]:

bill_methods = LabelEncoder().fit(df["avg_bill_methods"].to_numpy().reshape(-1, 1))
gender = LabelEncoder().fit(df["collector_gender"].to_numpy().reshape(-1, 1))
mart = LabelEncoder().fit(df["collector_marital_status"].to_numpy().reshape(-1, 1))
nego = LabelEncoder().fit(df["negotiation_skills"].to_numpy().reshape(-1, 1))
df2 = pd.DataFrame()

df2["avg_bill_methods"] = bill_methods.transform(df["avg_bill_methods"])
df2["collector_gender"] = gender.transform(df["collector_gender"])
df2["collector_marital_status"] = mart.transform(df["collector_marital_status"])
df2["negotiation_skills"] = nego.transform(df["negotiation_skills"])

scaler = RobustScaler().fit(df[["bill_amount_collected", "total_actual"]])
df2[["bill_amount_collected", "total_actual"]] = scaler.transform(df[["bill_amount_collected", "total_actual"]])
df2[["debtor_volume_handled", "collector_age", "success_rate", "collector_experience", "time_to_collect"]] = df[["debtor_volume_handled", "collector_age", "success_rate", "collector_experience", "time_to_collect"]]
y = df["total_cost"]

df2.head(5)

,avg_bill_methods,collector_gender,collector_marital_status,negotiation_skills,bill_amount_collected,total_actual,debtor_volume_handled,collector_age,success_rate,collector_experience,time_to_collect
0,2,0,3,0,-0.46,-0.52,7,45,40.00,4,0
1,1,1,2,1,-0.28,-0.34,35,32,49.00,0,11
2,2,0,1,1,-0.48,-0.53,6,62,27.00,0,4
3,0,1,2,2,1.64,1.39,63,55,56.00,2,29
4,0,1,3,2,0.45,0.11,35,53,71.00,2,22


In [7]:
X = df2
y_ = RobustScaler().fit(y.to_numpy().reshape(-1, 1))
y = y_.transform(y.to_numpy().reshape(-1, 1))
y = pd.Series(y.ravel())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
reg = LazyRegressor(verbose=0, ignore_warnings=True)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

  5%|▍         | 2/42 [00:00<00:04,  8.08it/s]

100%|██████████| 42/42 [00:06<00:00,  6.31it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 768
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 11
[LightGBM] [Info] Start training from score 0.296629
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
AdaBoostRegressor,0.67,0.68,0.40,0.08
RandomForestRegressor,0.65,0.67,0.41,1.37
ExtraTreesRegressor,0.65,0.67,0.41,0.86
BaggingRegressor,0.64,0.66,0.41,0.16
MLPRegressor,0.62,0.64,0.43,0.97
OrthogonalMatchingPursuitCV,0.62,0.64,0.43,0.02
LassoLarsIC,0.62,0.64,0.43,0.02
ElasticNetCV,0.62,0.64,0.43,0.29
LassoCV,0.62,0.64,0.43,0.12


In [8]:
y.describe()

count   1000.00
mean       0.32
std        0.68
min       -0.25
25%       -0.25
50%        0.00
75%        0.75
max        2.51
dtype: float64

In [9]:
y = y_.inverse_transform(y.to_numpy().reshape(-1, 1))
y = pd.Series(y.ravel())
y.describe()

count        1000.00
mean    139633105.12
std     166600490.67
min         25263.00
25%       1928856.50
50%      62215627.00
75%     247064723.00
max     678041154.00
dtype: float64